In [61]:
import os
import shutil

In [62]:
PACKET_LOSS_TOOL_DIR = "/home/dangnp/workspace/tools/loss-simulator/frontend/loss-simulator/public/audio"

OPUS_PLC_SAMPLE_DIR = "./audio-opusPlc"
NORMAL_SAMPLE_DIR = "./audio-normal"
SCORED_AUDIO_DIR = "./scored-audio"

NETWORK_TYPE = ["3g", "4g"]
LOSS_PERCENTAGE = ["0", "10", "30", "50", "70", "90"]
AUDIO_FILE_NAME = ["opusPlc", "normal"]
AUDIO_FILE_TYPE = ".wav"

In [63]:
shutil.copytree(PACKET_LOSS_TOOL_DIR, OPUS_PLC_SAMPLE_DIR, dirs_exist_ok=True)

'./audio-opusPlc'

In [64]:
shutil.copytree(PACKET_LOSS_TOOL_DIR, NORMAL_SAMPLE_DIR, dirs_exist_ok=True)

'./audio-normal'

In [65]:
!rm -rf scored-audio
os.makedirs(SCORED_AUDIO_DIR, exist_ok=True)

### Create audio folder

Generation function

In [66]:
def genAudioFiles(audioType = AUDIO_FILE_NAME[0]):
    counter = 0
    if audioType == AUDIO_FILE_NAME[0]:
        baseDir = OPUS_PLC_SAMPLE_DIR
    else:
        baseDir = NORMAL_SAMPLE_DIR
    sampleFolder = os.listdir(baseDir)
    sampleFolder.sort()
    for folder in sampleFolder:
        print(f"Processing {folder} ...")
        folderPath = baseDir + "/" + folder
        target = folderPath + "/" + [file for file in os.listdir(folderPath) if file.startswith("speaker")][0]
        network = NETWORK_TYPE[counter // len(LOSS_PERCENTAGE)]
        loss = LOSS_PERCENTAGE[counter % len(LOSS_PERCENTAGE)]
        dest = SCORED_AUDIO_DIR + "/" + f"{network}-loss{loss}"
        filename = audioType + AUDIO_FILE_TYPE
        os.makedirs(dest, exist_ok=True)

        shutil.copyfile(target, dest + "/" + filename)
        counter += 1

Start generating

In [67]:
genAudioFiles(AUDIO_FILE_NAME[0])


Processing 26-09-2025_123058 ...
Processing 26-09-2025_123136 ...
Processing 26-09-2025_123213 ...
Processing 26-09-2025_123251 ...
Processing 26-09-2025_123328 ...
Processing 26-09-2025_123406 ...
Processing 26-09-2025_123443 ...
Processing 26-09-2025_123521 ...
Processing 26-09-2025_123558 ...
Processing 26-09-2025_123635 ...
Processing 26-09-2025_123712 ...
Processing 26-09-2025_123750 ...


In [69]:
genAudioFiles(AUDIO_FILE_NAME[1])

Processing 26-09-2025_122027 ...


IndexError: list index out of range

### PLC_MOS Scoring

In [70]:
import soundfile as sf
import pandas as pd
from plc_mos import PLCMOSEstimator

In [71]:
plcmos = PLCMOSEstimator()
data, sr = sf.read("./scored-audio/3g-loss10/opusPlc.wav")
mos = plcmos.run(data, sr)
mos

1.9935580809911093

In [72]:
rows = []
plc = []
normal = []

In [73]:
for networkType in os.listdir(SCORED_AUDIO_DIR):
    rows.append(networkType)
    networkPath = SCORED_AUDIO_DIR + "/" + networkType
    
    plcFilePath = networkPath + "/" + AUDIO_FILE_NAME[0] + AUDIO_FILE_TYPE
    data, sr = sf.read(plcFilePath)
    mos = plcmos.run(data, sr)
    plc.append(mos)

    normalFilePath = networkPath + "/" + AUDIO_FILE_NAME[1] + AUDIO_FILE_TYPE
    data, sr = sf.read(normalFilePath)
    mos = plcmos.run(data, sr)
    normal.append(mos)

In [ ]:
df = pd.DataFrame({"Network Condition": rows, AUDIO_FILE_NAME[0]: plc, AUDIO_FILE_NAME[1]: normal, "diff (opusPlc - normal)": (pd.Series(plc) - pd.Series(normal))})
df = df.set_index("Network Condition")
df = df.sort_index()
df.to_excel("output.xlsx", sheet_name="Complexity5")
df

,opusPlc,normal,diff (opusPlc - normal)
Network Condition,,,
3g-loss0,2.659453,2.388245,0.271207
3g-loss10,1.993558,2.225534,-0.231976
3g-loss30,1.554802,1.575320,-0.020519
3g-loss50,1.444333,1.408174,0.036159
3g-loss70,1.290830,1.274282,0.016549
3g-loss90,1.204488,1.139217,0.065272
4g-loss0,2.688106,2.313467,0.374638
4g-loss10,2.383453,2.217185,0.166268
4g-loss30,2.213061,2.097387,0.115675
